[1] Package import

In [26]:
import os


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

[2] functions   

In [27]:
#activation function
#sigmoid function
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)


[3] read data

In [28]:
file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# make a matrix each column of which represents an images in a vector form 
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector    

    count += 1
    
    
#Datanum
trDataNum=1000
tDataNum=9000
    
trainX = list_image[:,0:1000] 
trainY = list_label[0:1000] 
testX =  list_image[:,1000:10000] 
testY = list_label[1000:10000]

nb_classes = 10

trainY =np.eye(nb_classes)[trainY]
testY = np.eye(nb_classes)[testY]
                          
trainY = trainY.T
testY = testY.T

[4] initailize values

In [29]:
# Epoch num
NUM_EPOCH=15000

#Array for writing loss, accuracy
trLossArray=np.zeros((NUM_EPOCH+1,1))
tLossArray=np.zeros((NUM_EPOCH+1,1))
trAcArray=np.zeros((NUM_EPOCH+1,1))
tAcArray=np.zeros((NUM_EPOCH+1,1))

lr=0.2 #learning Rate

#weight

# np.random.seed(0)

layer_num = 3

u=np.random.randn(196,784)#layer 1 weight (input : input feature) 

v=np.random.randn(49,196)#layer 2 weight (input: w1 output feature) 

w=np.random.randn(10,49)#layer 3  weight (input :w2 output feature) 

#bias 
a=np.zeros((1,1))#layer 1  bias 

b=np.zeros((1,1))#layer 2 bias 

c=np.zeros((1,1))#layer 3 bias 

#X for drawing the graph
x_=np.arange(NUM_EPOCH+1)#X

#ramda
ramda = 0.001

[5] training(gradient decent)

In [ ]:
for epoch in range(NUM_EPOCH+1):
    
    #forward propagation(train)
    trZ1=np.dot(u,trainX)+a#Layer 1
    trA1=sigmoid(trZ1)

    trZ2=np.dot(v,trA1)+b#Layer 2
    trA2=sigmoid(trZ2)
    
    trZ3=np.dot(w,trA2)+c#Layer 3
    trA3=sigmoid(trZ3)

    #get train loss
    trloss=-(np.multiply(trainY,np.log(trA3))+np.multiply((1-trainY),np.log(1-trA3)))
    trloss=np.mean(trloss.T,axis=1)
    trloss=1/trDataNum*np.sum(trloss) + ramda/(2*layer_num)*(np.power(u,2).sum() + np.power(v,2).sum() + np.power(w,2).sum())
    trLossArray[epoch]=trloss
    
    #forward propagation(test)
    tZ1=np.dot(u,testX)+a#Layer 1
    tA1=sigmoid(tZ1)
    
    tZ2=np.dot(v,tA1)+b#Layer 2
    tA2=sigmoid(tZ2)
    
    tZ3=np.dot(w,tA2)+c#Layer 3
    tA3=sigmoid(tZ3)
    
    #get test loss
    tloss=-(np.multiply(testY,np.log(tA3))+np.multiply((1-testY),np.log(1-tA3)))
    tloss=np.mean(tloss.T,axis=1)
    tloss=1/tDataNum*np.sum(tloss) + ramda/(2*layer_num)*(np.power(u,2).sum() + np.power(v,2).sum() + np.power(w,2).sum())
    tLossArray[epoch]=tloss
    
    #get Accuracy
    trainPY1=np.argmax(trA3.T,axis=1)
    trainPY2=np.argmax(trainY.T,axis=1)
    trAccuracy=((trainPY1 == trainPY2).sum())/trDataNum
    trAcArray[epoch]=trAccuracy
    
    testPY1 = np.argmax(tA3.T,axis=1)
    testPY2 = np.argmax(testY.T,axis=1)
    tAccuracy=((testPY1 == testPY2).sum())/tDataNum
    tAcArray[epoch]=tAccuracy
    
    #backward propagation
    dz3=trA3-trainY
    dw=1/trDataNum*np.dot(dz3,trA2.T)
    dc=1/trDataNum*np.sum(dz3,axis=1,keepdims=True)
    
    dz2=np.multiply(np.dot(w.T,dz3),trA2*(1.0-trA2))
    dv=1/trDataNum*np.dot(dz2,trA1.T)
    db=1/trDataNum*np.sum(dz2,axis=1,keepdims=True)
    
    dz1=np.multiply(np.dot(v.T,dz2),trA1*(1.0-trA1))
    du=1/trDataNum*np.dot(dz1,trainX.T)
    da=1/trDataNum*np.sum(dz1,axis=1,keepdims=True)
    
    
    #update weight and bias
    u=(1-ramda*lr)*u-lr*du
    a=(1-ramda*lr)*a-lr*da    

    v=(1-ramda*lr)*v-lr*dv
    b=(1-ramda*lr)*b-lr*db
    
    w=(1-ramda*lr)*w-lr*dw
    c=(1-ramda*lr)*c-lr*dc
    
    
#     #check the data per epoch 50
#     if epoch%500==0:    
#         print("epoch :" + str(epoch+1))
#         print("train loss :  " +np.array2string(trloss))
#         print("test loss :  " +np.array2string(tloss))
#         print("train accuracy :  " +np.array2string(trAccuracy))
#         print("test accuracy :  " +np.array2string(tAccuracy))


[6] find correct and wrong 10 images

In [ ]:
## find correct Image

count= 0
correctNum = 0
correctImage = np.empty((784,10))
correctLabel = np.empty(10)
while(1):
    
    if(correctNum==10):
        break
    
    tZ1=np.dot(u,testX[:,count:count+1])+a#Layer 1
    tA1=sigmoid(tZ1)
    
    tZ2=np.dot(v,tA1)+b#Layer 2
    tA2=sigmoid(tZ2)
    
    tZ3=np.dot(w,tA2)+c#Layer 3
    tA3=sigmoid(tZ3)

    PY1=np.argmax(tA3.T)
    PY2=np.argmax(testY[:,count:count+1].T)
    if PY1==PY2:
        correctImage[:,correctNum] = testX[:,count]
        correctLabel[correctNum] = PY1
        correctNum = correctNum + 1
        count = count + 1
        
    else: 
        count = count + 1

## find wrong Image
        
count= 0
wrongNum = 0
wrongImage = np.empty((784,10))
wrongLabel = np.empty(10)
while(1):
    
    if(wrongNum==10):
        break
    
    tZ1=np.dot(u,testX[:,count:count+1])+a#Layer 1
    tA1=sigmoid(tZ1)
    
    tZ2=np.dot(v,tA1)+b#Layer 2
    tA2=sigmoid(tZ2)
    
    tZ3=np.dot(w,tA2)+c#Layer 3
    tA3=sigmoid(tZ3)

    PY1=np.argmax(tA3.T)
    PY2=np.argmax(testY[:,count:count+1].T)
    
    if PY1!=PY2:
        wrongImage[:,wrongNum] = testX[:,count]
        wrongLabel[wrongNum] = PY1
        wrongNum = wrongNum + 1
        count = count + 1
        
    else: 
        count = count + 1

[7] Plot the loss curve

In [ ]:
plt.title("train loss and test loss")
plt.scatter(x_,trLossArray ,c="b",)#train loss 
plt.scatter(x_,tLossArray ,c="r")#test loss
plt.show()

[8] Plot the accuracy curve

In [ ]:
plt.title("train Ac and test Ac")
plt.scatter(x_,trAcArray,c="b")#train ac 
plt.scatter(x_,tAcArray,c="r")#test ac 
plt.show()

[9] Plot the accuracy value

In [ ]:
print("Final train accuracy :  " +np.array2string(trAcArray[NUM_EPOCH]))
print("Final test accuracy :  " +np.array2string(tAcArray[NUM_EPOCH]))

[10] Plot the classification example

In [ ]:
print("correct 10 images")
f1 = plt.figure(1)

for i in range(10):

    label       = int(correctLabel[i])
    im_vector   = correctImage[:,i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()

In [ ]:
print("wrong 10 images")
f2 = plt.figure(1)

for i in range(10):

    label       = int(wrongLabel[i])
    im_vector   = wrongImage[:,i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()